In [1]:
# Regular EDA(exploratory data analysis) and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

from tqdm import tqdm
import string
import statistics
import re
import joblib
import random

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import the training set
df = pd.read_csv('/kaggle/input/dataverse-ipa/dataverse_2023/trainIPAdb_u.csv')
df.head(2)

,text,ipa
0,এরপরও তারা বকেয়া পরিশোধ করেনি।,eɾpɔɾo t̪ɐɾɐ bɔkeʲɐ poɾɪʃod̪ʱ kɔɾenɪ।
1,আগে সুইস ব্যাংকে জমা টাকার কোনো প্রতিবেদন প্রক...,ɐge suɪ̯s bɛŋke ɟɔmɐ tɐkɐɾ kono pɾot̪ɪbed̪ɔn p...


In [3]:
# See a random sample from the training set 
rand_index = np.random.randint(0, len(df))
print(f'Text = {df.text[rand_index]}\nIPA = {df.ipa[rand_index]}')

Text = দীর্ঘদিন বাচসাস-এর ভোট না হওয়ায় এ নির্বাচনকে বেশ গুরুত্বের সঙ্গেই দেখছেন ভোটাররা।
IPA = d̪ɪɾgʱod̪ɪn bɐcʃɐʃ-eɾ bʱot nɐ hoʷɐe̯ e nɪɾbɐcɔnke beʃ guɾut̪t̪eɾ ʃɔŋgeɪː d̪ekʰcʰen bʱotɐɾɾɐ।


In [4]:
new_df = pd.read_csv('/kaggle/input/dataverse-ipa/new_train.csv')
new_df.head(3)

,text,ipa
0,আমি বলেন সাত আদালতের।,ɐmɪ bɔlen ʃɐt̪ ɐd̪ɐlɔt̪eɾ।
1,তুমি করা চুরানব্বই রাখে।,t̪umɪ kɔɾɐ cuɾɐnɔbboɪ ɾɐkʰe।
2,আমরা স্বীকার আটান্ন উপস্থিত।,ɐmɾɐ ʃɪkɐɾ ɐtɐnno upost̪ʰɪt̪।


In [5]:
test_df = pd.read_csv('/kaggle/input/dataverse-ipa/dataverse_2023/testData.csv')
test_df.head(3)

,row_id_column_name,text
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।


## Training Data Library 

In [6]:
# Make a function
def make_library(df):
    '''
    
    '''
    library = {}
    texts = df.text
    ipas = df.ipa
    for i in tqdm(range(len(df))):
        for (v1, v2) in zip(re.split(r'\s+|(?=।)', texts[i]), re.split(r'\s+|(?=।)', ipas[i])):
            library[v1] = v2
    return library

In [7]:
lib = make_library(df)
# lib.update(make_library(new_df))

100%|██████████| 21999/21999 [00:00<00:00, 46877.12it/s]


In [8]:
# Add brackets is the library
lib['('] = '('
lib[')'] = ')'

In [9]:
print(list(lib.items())[:10])
len(lib)

[('এরপরও', 'eɾpɔɾo'), ('তারা', 't̪ɐɾɐ'), ('বকেয়া', 'bɔkeʲɐ'), ('পরিশোধ', 'poɾɪʃod̪ʱ'), ('করেনি', 'kɔɾenɪ'), ('।', '।'), ('আগে', 'ɐge'), ('সুইস', 'suɪ̯s'), ('ব্যাংকে', 'bɛŋke'), ('জমা', 'ɟɔmɐ')]


34817

In [10]:
bengali_char_pattern = re.compile(r'[\u0980-\u09FF]')

filtered_lib = {key: value for key, value in lib.items() if not re.search(bengali_char_pattern, value)}

print("Filtered Dictionary:", len(filtered_lib))

Filtered Dictionary: 34505


In [11]:
print(list(filtered_lib.items())[:10])
len(filtered_lib)

[('এরপরও', 'eɾpɔɾo'), ('তারা', 't̪ɐɾɐ'), ('বকেয়া', 'bɔkeʲɐ'), ('পরিশোধ', 'poɾɪʃod̪ʱ'), ('করেনি', 'kɔɾenɪ'), ('।', '।'), ('আগে', 'ɐge'), ('সুইস', 'suɪ̯s'), ('ব্যাংকে', 'bɛŋke'), ('জমা', 'ɟɔmɐ')]


34505

In [12]:
try:
    lib['শ্রেণী']
except:
    print('ValueError')

In [13]:
lib = filtered_lib
len(lib)

34505

### Modeling

In [14]:
from transformers import AutoTokenizer
model_id = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [15]:
# Later, to load the model back
loaded_model = joblib.load('/kaggle/input/model1/model (1).pkl')
loaded_model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [16]:
# # Remove English alphanumeric values
# alpha_pat = "[a-zA-z0-9০-৯()]"
# test_df["text"] = test_df["text"].str.replace(alpha_pat, "", regex=True)

In [17]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model=loaded_model,tokenizer=tokenizer, device=0)

In [18]:
for i in range(3):
    texts = "সুন্দর সুন্দরমনা আমি তোমাকে ভালবাসি।"
    # texts = "বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশনাল ডাইরেক্টরগেমস অ্যান্ড স্পোর্টস ডিপার্টমেন্ট ওয়ালটন।"
    # texts = 'আমি তোমাকে মন।'
    ipas = pipe(texts, max_length=128, batch_size=16)
print(ipas)

[{'generated_text': 'ʃund̪oɾ ʃund̪ɔɾmonɐ ɐmɪ t̪omɐke bɦɐlobɐʃɪ।'}]


In [19]:
def calculate_ipa(txt):
    '''
    Converts a sentence to IPA using t5 model.
    '''
    ipas = pipe(txt, max_length=128, batch_size=16)
    return ipas[0]['generated_text']

def word_to_ipa(word):
    '''
    Converts a Word to IPA using t5 model.
    '''
    word += '।'
    ipas = pipe(word, max_length=128, batch_size=16)
    output_string = ' '.join(ipas[0]['generated_text'].split())
    return output_string[:-1]

In [20]:
# Check the function
word_to_ipa('তারা-'), word_to_ipa('তোমাকে,')
word_to_ipa('তারা-'), word_to_ipa('তোমাকে,')

('t̪ɐɾɐ-', 't̪omɐke,')

In [21]:
word_to_ipa('শ্রেণীর')

'শ্রেণির'

In [22]:
def contains_bengali_digit(text):
    '''
    Is there a bengali digit in text
    '''
    return bool(re.search(r'[০-৯]', text))

In [23]:
def ipa_converter(text):
    '''
    Converts a sentence to IPA
    '''
    ipa = ''
    #for value in re.split(r'\s+|(?=।)', text):
    words = re.split(r'\s+|(?<=[()।])|(?=[()।])', text)
    
    # Filter out empty strings and extra spaces from the result
    words = [word.strip() for word in words if word.strip()]
    for value in words:
        if value in lib:
            ipa += lib[value]
            ipa += ' '
        else:
            # ipa += 'UNKey '
            ipa += word_to_ipa(value)
            ipa += ' '
            
    pattern = r'\s+(?=।)|(?<=।)\s+'
    cln_ipa = re.sub(pattern, '', ipa)
    
    # Replace ' (' with '('
    cln_ipa = re.sub(r'\(\s+', '(', cln_ipa)

    # Replace ') ' with ')'
    cln_ipa = re.sub(r'\s+\)', ')', cln_ipa)
    return cln_ipa

In [24]:
txt = 'তার সারা শ্রেণীর অচিন্ত শরীরে ভোঁতা অস্ত্রের আটত্রিশটি আঘাত রয়েছে।'
ipa_converter(txt)

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


't̪ɐɾ ʃɐɾɐ শ্রেণির ɔcɪnt̪o ʃoɾɪɾe bɦõt̪ɐ ɔst̪ɾeɾ ɐtt̪ɾɪʃtɪ ɐgʱɐt̪ ɾoʲecʰe।'

In [25]:
lib['বাংলাদেশ']

'bɐŋlɐd̪eʃ'

In [26]:
rand_index = np.random.randint(0, 1000)
print(rand_index)
print(f'Pred = {ipa_converter(test_df.text[rand_index])}\nTrue = {test_df.text[rand_index]}')

127
Pred = ɟɪnɪ ʃɐle bɦɪʃɪɐ̯ɾ ɟɐt̪ɾɐ ʃuɾu koɾleo̯, pɔɾoboɾt̪ɪ ʃɔmɔʲe dɪbɦɪdɪ pɾoɟukt̪ɪ bɐɟɐɾe eɾ ɟɐʲgɐ d̪ɔkʰl koɾe nɛe̯।
True = ১৯৭০ সালে ভিসিআর যাত্রা শুরু করলেও, পরবর্তী সময়ে ডিভিডি প্রযুক্তি বাজারে এর জায়গা দখল করে নেয়।


## WORD ERROR RATE 

In [27]:
txt = '১৯ নভেম্বর নরেন্দ্র মোদি স্টেডিয়াম, বাংলাদেশ বিশ্বকাপ শিরোপা তুললো।'
actual = 'unɪʃ novembɔɾ nɔɾendɾɐ modɪ stedɪum, bɐŋlɐd̪eʃ bɪʃʃokɐp ʃɪɾopɐ t̪ullo।'
ipa_converter(txt)

't̪ɪn nɔbʱembɔɾ nɔɾend̪ɾo mod̪ɪ stedɪjɐm, bɐŋlɐd̪eʃ bɪʃʃokɐp ʃɪɾopɐ t̪olo।'

# **Need to handle Exceptional Cases**
1. Numeric Words (ie. ০-৯)  ***(Done)***
2. Bracket      ***(Done)***
3. হয়’ --> hɔj ‘    
4. Mistake in Library (ie. শ্রেণির)   ***(Done)***
5. dict_change  =  "bɦ"  : "bʱ",   "dɦ":"dʱ"   etc
    

### BRACKETS HANDLING

In [28]:
# Add brackets is the library
lib['('] = '('
lib[')'] = ')'

In [29]:
# See a random sample from the test set 
rand_index = np.random.randint(0, len(test_df))
print(f'Text = {test_df.text[rand_index]}')

Text = এছাড়া মহাপরিচালকের সংখ্যা ৬ থেকে বাড়িয়ে ৮ জন করার প্রস্তাব করা হয়েছে।


In [30]:
import re

# Input sentence
text = "মেয়েদের আনইভেন বার (রাত ১টা ২১)ভারোত্তোলনমেয়েদের ৭৫ কেজি ফাইনাল (রাত ১২টা ৩০ ও ৪টা)ডাইভিংমেয়েদের ৩ মিটার স্প্রিং বোর্ড (রাত ১টা)।"

# Split the sentence using a regular expression to keep spaces, periods, and parentheses
words = re.split(r'\s+|(?<=[()।])|(?=[()।])', text)

# Filter out empty strings and extra spaces from the result
words = [word.strip() for word in words if word.strip()]

# Print the words
print(words)

['মেয়েদের', 'আনইভেন', 'বার', '(', 'রাত', '১টা', '২১', ')', 'ভারোত্তোলনমেয়েদের', '৭৫', 'কেজি', 'ফাইনাল', '(', 'রাত', '১২টা', '৩০', 'ও', '৪টা', ')', 'ডাইভিংমেয়েদের', '৩', 'মিটার', 'স্প্রিং', 'বোর্ড', '(', 'রাত', '১টা', ')', '।']


In [31]:
ipa_converter(text)

'mejed̪eɾ ɐnɪbɦen bɐɾ (ɾɐt̪ dɛtɐ bɪʃʃo) bɦɐɾot̪t̪olonomejed̪eɾ ʃɐbbo keɟɪ pʰɐɪ̯nɐl (ɾɐt̪ bɪstɐ pɾo o tɪntɐ) dɐɪ̯bɦɪŋkombed̪eɾ t̪uɪ̯ mɪtɐɾ spoɾɪŋ boɾd (ɾɐt̪ d̪ɛtɐ)।'

### Search for exceptions 

In [32]:
filt_cond = ["১-৯", 'A-Za-z0-9']
# Filtering text samples that contain English alphanumeric values
filt = test_df[lambda x: x["text"].str.contains("[,]")]
filt = filt.reset_index(drop=True)
print(f'Length of the Df = {filt.shape}')

Length of the Df = (5279, 2)


In [33]:
ind = np.random.randint(0, len(filt))
filt.text[ind]

'খবর ডন অনলাইন, দ্য এক্সপ্রেস ট্রিবিউন, বিবিসির।'

In [34]:
ipa_converter(filt.text[ind])

'kʰɔboɾ dɔn ɔnlɐɪ̯n, ɔnlɐɪ̯n, d̪ɛ ekspɾes tɾɪbɪn, bɪbɪsɪɾ।'

## Handle the Number Case

In [35]:
import math
numeric_words = {
    '.': 'দশমিক',
    '0': '',
    '1': 'এক',
    '01': 'এক',
    '2': 'দুই',
    '02': 'দুই',
    '3': 'তিন',
    '03': 'তিন',
    '4': 'চার',
    '04': 'চার',
    '5': 'পাঁচ',
    '05': 'পাঁচ',
    '6': 'ছয়',
    '06': 'ছয়',
    '7': 'সাত',
    '07': 'সাত',
    '8': 'আট',
    '08': 'আট',
    '9': 'নয়',
    '09': 'নয়',
    '10': 'দশ',
    '11': 'এগারো',
    '12': 'বার',
    '13': 'তের',
    '14': 'চৌদ্দ',
    '15': 'পনের',
    '16': 'ষোল',
    '17': 'সতের',
    '18': 'আঠার',
    '19': 'উনিশ',
    '20': 'বিশ',
    '21': 'একুশ',
    '22': 'বাইশ',
    '23': 'তেইশ',
    '24': 'চব্বিশ',
    '25': 'পঁচিশ',
    '26': 'ছাব্বিশ',
    '27': 'সাতাশ',
    '28': 'আঠাশ',
    '29': 'ঊনত্রিশ',
    '30': 'ত্রিশ',
    '31': 'একত্রিশ',
    '32': 'বত্রিশ',
    '33': 'তেত্রিশ',
    '34': 'চৌত্রিশ',
    '35': 'পঁয়ত্রিশ',
    '36': 'ছত্রিশ',
    '37': 'সাঁইত্রিশ',
    '38': 'আটত্রিশ',
    '39': 'ঊনচল্লিশ',
    '40': 'চল্লিশ',
    '41': 'একচল্লিশ',
    '42': 'বিয়াল্লিশ',
    '43': 'তেতাল্লিশ',
    '44': 'চুয়াল্লিশ',
    '45': 'পঁয়তাল্লিশ',
    '46': 'ছেচল্লিশ',
    '47': 'সাতচল্লিশ',
    '48': 'আটচল্লিশ',
    '49': 'ঊনপঞ্চাশ',
    '50': 'পঞ্চাশ',
    '51': 'একান্ন',
    '52': 'বায়ান্ন',
    '53': 'তিপ্পান্ন',
    '54': 'চুয়ান্ন',
    '55': 'পঞ্চান্ন',
    '56': 'ছাপ্পান্ন',
    '57': 'সাতান্ন',
    '58': 'আটান্ন',
    '59': 'ঊনষাট',
    '60': 'ষাট',
    '61': 'একষট্টি',
    '62': 'বাষট্টি',
    '63': 'তেষট্টি',
    '64': 'চৌষট্টি',
    '65': 'পঁয়ষট্টি',
    '66': 'ছেষট্টি',
    '67': 'সাতষট্টি',
    '68': 'আটষট্টি',
    '69': 'ঊনসত্তর',
    '70': 'সত্তর',
    '71': 'একাত্তর',
    '72': 'বাহাত্তর',
    '73': 'তিয়াত্তর',
    '74': 'চুয়াত্তর',
    '75': 'পঁচাত্তর',
    '76': 'ছিয়াত্তর',
    '77': 'সাতাত্তর',
    '78': 'আটাত্তর',
    '79': 'ঊনআশি',
    '80': 'আশি',
    '81': 'একাশি',
    '82': 'বিরাশি',
    '83': 'তিরাশি',
    '84': 'চুরাশি',
    '85': 'পঁচাশি',
    '86': 'ছিয়াশি',
    '87': 'সাতাশি',
    '88': 'আটাশি',
    '89': 'ঊননব্বই',
    '90': 'নব্বই',
    '91': 'একানব্বই',
    '92': 'বিরানব্বই',
    '93': 'তিরানব্বই',
    '94': 'চুরানব্বই',
    '95': 'পঁচানব্বই',
    '96': 'ছিয়ানব্বই',
    '97': 'সাতানব্বই',
    '98': 'আটানব্বই',
    '99': 'নিরানব্বই',
    '100': 'একশো',
}

units = {
    'koti': 'কোটি',
    'lokkho': 'লক্ষ',
    'hazar': 'হাজার',
    'sotok': 'শো',
    'ekok': '',
}


def input_sanitizer(number):
    if isinstance(number, float) or isinstance(number, int) or \
            isinstance(number, str):
        if isinstance(number, str):
            try:
                if "." in number:
                    number = float(number)
                else:
                    number = int(number)
            except ValueError:
                return None
        return number
    else:
        return None


def generate_segments(number):
    """
    Generating the unit segments such as koti, lokkho
    """
    segments = dict()
    segments['koti'] = math.floor(number/10000000)
    number = number % 10000000
    segments['lokkho'] = math.floor(number/100000)
    number = number % 100000
    segments['hazar'] = math.floor(number/1000)
    number = number % 1000
    segments['sotok'] = math.floor(number/100)
    number = number % 100
    segments['ekok'] = number

    return segments


def float_int_extraction(number):
    """
    Extracting the float and int part from the passed number. The first return
    is the part before the decimal point and the rest is the fraction.
    """
    _number = str(number)
    if "." in _number:
        return tuple([int(x) for x in _number.split(".")])
    else:
        return number, None


def whole_part_word_gen(segments):
    """
    Generating the bengali word for the whole part of the number
    """
    generated_words = ''
    for segment in segments:
        if segments[segment]:
           
            if segment=="sotok":
                generated_words += numeric_words[str(segments[segment])] + \
                                   ""+ units[segment] + " "
            else:
                generated_words += numeric_words[str(segments[segment])] + \
                " " + units[segment] + " "

    return generated_words[:-2]


def fraction_to_words(fraction):
    """
    Generating bengali words for the part after the decimal point
    """
    generated_words = ""
    for digit in str(fraction):
        generated_words += numeric_words[digit] + " "
    return generated_words[:-1]


def to_bn_word(number):
    """
    Takes a number and outputs the word form in Bengali for that number.
    """
    if len(number)>9:
        return 'এক'
    generated_words = ""
    number = input_sanitizer(number)

    whole, fraction = float_int_extraction(number)

    whole_segments = generate_segments(whole)

    generated_words = whole_part_word_gen(whole_segments)

    if fraction:
        if generated_words:
            return generated_words + " দশমিক " + fraction_to_words(fraction)
        else:
            return "দশমিক " + fraction_to_words(fraction)
    else:
        return generated_words

In [36]:
num_dict={}
def num_to_text(string ):
    temp=[]
    pattern_to_keep = r'\d+'
    m=re.split("[অ-হ]|-| ",string)
    for k in m :
        if re.search('[১-৯]',k):
            result = re.sub(f'(?:(?!{pattern_to_keep}).)*', '', k)
            temp.append(result)
    
    if len(temp)<1:
        return string
    for i in temp:
        if i in num_dict:
            string=re.sub(i,num_dict[i],string)
        else:
            num_dict[i]=to_bn_word(i)
            string=re.sub(i,num_dict[i],string)
    return string

In [37]:
filt_cond = ["১-৯", 'A-Za-z0-9']
# Filtering text samples that contain English alphanumeric values
filt = test_df[lambda x: x["text"].str.contains("[১-৯]")]
filt = filt.reset_index(drop=True)
print(f'Length of the Df = {filt.shape}')

Length of the Df = (9601, 2)


In [38]:
ind = np.random.randint(0, len(filt))
print(ipa_converter(filt.text[ind]))
filt.text[ind]

mɔnt̪ɾɪ bɔlen, bɔɾt̪omɐne beʃɔɾkɐɾɪ poɾɟɐʲe cɪkɪt̪ʃɐʃebɐ kɐɾɟokɾom ‘d̪o medɪkel pɾɐktɪʃ ɛnd pɾɐɪ̯vet klɪnɐɾ ɛnd lɛboɾetɪɾɪɟ (ɾegulo) ɔɾdɪnnɔnʃo-chonɪnʃo-উত্য--ɟoɾdɪnɔnʃo-thɐŋkɪnʃo-ɟɪŋʃo-ɟo onuɟɐʲɪ poɾɪcɐlɪt̪o hoccʰe।


'মন্ত্রী বলেন, বর্তমানে বেসরকারি পর্যায়ে চিকিৎসাসেবা কার্যক্রম ‘দ্য মেডিকেল প্র্যাকটিস অ্যান্ড প্রাইভেট ক্লিনিক অ্যান্ড ল্যাবরেটরিজ (রেগুলেশন) অর্ডিন্যান্স-১৯৮২’ অনুযায়ী পরিচালিত হচ্ছে।'

In [39]:
num_to_text('রয়েছে')

'রয়েছে'

###  VIP Cell 

In [40]:
filt['ctext'] = filt['text'].apply(num_to_text)

###  Change *The Test Set Numbers to Words*

In [41]:
tqdm.pandas()
test_df['ctext'] = test_df['text'].progress_apply(num_to_text)

100%|██████████| 27228/27228 [00:01<00:00, 26635.31it/s]


In [42]:
test_df.head(2)

,row_id_column_name,text,ctext
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...,এ নিয়ে বিবাদে দুই হাজার পনের সালের দুই জুন রাত...


In [43]:
ind = np.random.randint(0, len(filt))
print(ipa_converter(filt.ctext[ind]))
filt.ctext[ind]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bɪd̪d̪ut̪ bɪbʱɐgeɾ konpbɦɐɾʃɔn ɔbo sɪlet ɛkʃo pɔncɐʃ megɐo̯jɐt tu d̪uɪ̯ʃo põcɪʃ megɐo̯wɐt kɔmbɐ͡ɪ̯nd sɐɪ̯kel pɐo̯ʷɐɾ plɛnd pɾokɔlpotɪ d̪uɪ̯ hɐɟɐɾ t̪ɛɾo ʃɐle gɾohon kɔɾɐ hɔe̯।


'বিদ্যুৎ বিভাগের কনভারশন অব সিলেট একশো পঞ্চাশ মেগাওয়াট টু দুইশো পঁচিশ মেগাওয়াট কম্বাইন্ড সাইকেল পাওয়ার প্ল্যান্ট প্রকল্পটি দুই হাজার তের সালে গ্রহণ করা হয়।'

In [44]:
# lib['শ্রেণীর']

### Handle This Library Error

In [45]:
y = list(lib.keys())
x = list(lib.values())
c  = 0
bengali_char_pattern = re.compile(r'[\u0980-\u09FF]')
for k,v in zip(y, x):
    if re.search(bengali_char_pattern,v):
        print(f'found {k}  {v} ')
        c += 1
        if c > 9:
            break
print(c)

0


In [46]:
txt = 'তার সারা শ্রেণীর অচিন্ত শরীরে ভোঁতা অস্ত্রের আটত্রিশটি আঘাত রয়েছে।'
ipa_converter(txt)

't̪ɐɾ ʃɐɾɐ শ্রেণির ɔncɪnt̪o ʃoɾɪɾe bɦõt̪ɐ ɔst̪ɾeɾ ɐtt̪ɾɪʃtɪ ɐgʱɐt̪ ɾoʲecʰe।'

In [47]:
len(lib)

34505

In [48]:
txt = 'তার সারা শ্রেণীর অচিন্ত শরীরে ভোঁতা অস্ত্রের আটত্রিশটি আঘাত রয়েছে।'
ipa_converter(txt)

't̪ɐɾ ʃɐɾɐ শ্রেণির ɔcɪnt̪o ʃoɾɪɾe bɦõt̪ɐ ɔst̪ɾeɾ ɐtt̪ɾɪʃtɪ ɐgʱɐt̪ ɾoʲecʰe।'

#### Fixed in the upper part

## Change this Part Idea and Code `Nannu`

In [49]:
dict_change={
    "bɦ":"bʱ",
    "dɦ":"dʱ",
    "oɦ":'oʱ',
    "nɦ":'nʱ',
    "ɟɦ":'ɟʱ',
    "ɽɦ":'ɽʱ',
    "gɦ":'gʱ',
    "tɦ":'tʱ',
     "d̪ɦ":'d̪ʱ',
     "j":"ʲ",
     "kh":"kʰ",
     "t̪h":"t̪ʰ",
    "ph":"pʰ",
    "ch":"cʰ",
    "w":"ʷ"
}

In [50]:
pattern = re.compile('|'.join(re.escape(key) for key in dict_change.keys()))

# Define a function to perform replacements
def replace(match):
    return dict_change[match.group(0)]

# Use re.sub() to perform replacements
def output(input_string):
    return pattern.sub(replace, input_string)
pattern

re.compile(r'bɦ|dɦ|oɦ|nɦ|ɟɦ|ɽɦ|gɦ|tɦ|d̪ɦ|j|kh|t̪h|ph|ch|w', re.UNICODE)

In [51]:
tst = 't̪ɐɾ ʃɐɾɐ শ্রেণির ɔcɪnt̪o ʃoɾɪɾe bɦõt̪ɐ ɔst̪ɾeɾ ɐtt̪ɾɪʃtɪ ɐgʱɐt̪ ɾoʲecʰe।'
output(tst)

't̪ɐɾ ʃɐɾɐ শ্রেণির ɔcɪnt̪o ʃoɾɪɾe bʱõt̪ɐ ɔst̪ɾeɾ ɐtt̪ɾɪʃtɪ ɐgʱɐt̪ ɾoʲecʰe।'

## Seems ok for the moment. Now Just Make a submission File

In [52]:
test_df.head()

,row_id_column_name,text,ctext
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...,এ নিয়ে বিবাদে দুই হাজার পনের সালের দুই জুন রাত...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।,আজ থেকে চৌদ্দ বছর আগে তিনি চলে গেছেন না ফেরার ...
3,3,নিহত ব্যক্তি কুতপালং টালের ই-২ ব্লকের আবুল বাছ...,নিহত ব্যক্তি কুতপালং টালের ই-দুই ব্লকের আবুল ব...
4,4,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস ৪৮২ (করুনা...,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস চারশো বিরা...


In [53]:
tqdm.pandas()
test_df['ipa'] = test_df['ctext'].progress_apply(ipa_converter)

  0%|          | 0/27228 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 27228/27228 [1:25:47<00:00,  5.29it/s]


In [54]:
tqdm.pandas()
test_df['fipa'] = test_df['ipa'].progress_apply(output)

100%|██████████| 27228/27228 [00:00<00:00, 215823.38it/s]


In [55]:
test_df.head()

,row_id_column_name,text,ctext,ipa,fipa
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,bɪʃeʃ ot̪ɪt̪ʰɪ ephem ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛd...,bɪʃeʃ ot̪ɪt̪ʰɪ epʰem ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛd...
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...,এ নিয়ে বিবাদে দুই হাজার পনের সালের দুই জুন রাত...,e nɪʲe bɪbɐd̪e d̪uɪ̯ hɐɟɐɾ pɔneɾo ʃɐleɾ d̪uɪ̯ ...,e nɪʲe bɪbɐd̪e d̪uɪ̯ hɐɟɐɾ pɔneɾo ʃɐleɾ d̪uɪ̯ ...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।,আজ থেকে চৌদ্দ বছর আগে তিনি চলে গেছেন না ফেরার ...,ɐɟ t̪ʰeke coud̪d̪o bɔcʰoɾ ɐge t̪ɪnɪ cɔle gɛcʰe...,ɐɟ t̪ʰeke coud̪d̪o bɔcʰoɾ ɐge t̪ɪnɪ cɔle gɛcʰe...
3,3,নিহত ব্যক্তি কুতপালং টালের ই-২ ব্লকের আবুল বাছ...,নিহত ব্যক্তি কুতপালং টালের ই-দুই ব্লকের আবুল ব...,nɪhɔt̪o bɛkt̪ɪ kut̪pɐlɔŋ tɐleɾ ɪ-d̪uɪ̯ blokeɾ ...,nɪhɔt̪o bɛkt̪ɪ kut̪pɐlɔŋ tɐleɾ ɪ-d̪uɪ̯ blokeɾ ...
4,4,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস ৪৮২ (করুনা...,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস চারশো বিরা...,ʃɔŋkkʰɪpt̪o skɾosɾɪlɔŋkɐ pɾot̪ʰom ɪnɪŋʃo cɐɾʃo...,ʃɔŋkkʰɪpt̪o skɾosɾɪlɔŋkɐ pɾot̪ʰom ɪnɪŋʃo cɐɾʃo...


## Check Manually

In [56]:
ri = np.random.randint(0, 100)
test_df['ipa'][ri], test_df['fipa'][ri]

("‘pɪthɐɾ ʃɐd̪e, ‘ ephbɪeʃeɾ mɐthe ‘ sɛgɐne dʱɐkɐ bɪʃʃobɪd̪d̪ɐlɔʲeɾ bɛbʃɐe̯ ʃɪkkʰɐ onuʃɔd̪ cʰɐt̪ɾolɪg ‘pɪthɐ ‘pɪthɐ ut̪ʃɔb d̪uɪ̯ hɐɟɐɾ ʃot̪eɾ ‘ʃɔt̪eɾ' eɾ ɐʲoɟon koɾe।",
 "‘pɪthɐɾ ʃɐd̪e, ‘ epʰbɪeʃeɾ mɐthe ‘ sɛgɐne dʱɐkɐ bɪʃʃobɪd̪d̪ɐlɔʲeɾ bɛbʃɐe̯ ʃɪkkʰɐ onuʃɔd̪ cʰɐt̪ɾolɪg ‘pɪthɐ ‘pɪthɐ ut̪ʃɔb d̪uɪ̯ hɐɟɐɾ ʃot̪eɾ ‘ʃɔt̪eɾ' eɾ ɐʲoɟon koɾe।")

In [57]:
test_df.to_csv('submission.csv', index=False)

In [58]:
# See a random sample from the test set 
rand_index = np.random.randint(0, len(test_df))
print(f'Text = {test_df.text[rand_index]}\nIpa = {test_df.ipa[rand_index]}')

Text = সোমবার আপিল বিভাগে তার রায় রিভিউ শুনানি এক মাস পিছিয়ে দেয়া হয়।
Ipa = ʃombɐɾ ɐpɪl bɪbʱɐge t̪ɐɾ ɾɐe̯ ɾɪbʱɪu̯ ʃunɐnɪ ɛk mɐʃ pɪcʰɪʲe d̪eʲɐ hɔe̯।


In [59]:
lib['মধ্যে']

'mod̪d̪ʱe'

In [60]:
# lenli = []
# for i in range(len(test_df)):
#     l1 = len(test_df['text'][i].split())
#     l2 = len(test_df['ipa'][i].split())
#     lenli.append([l1, l2])

In [61]:
# lenlisrt = sorted(lenli, key= lambda x:x[1], reverse=False)

In [62]:
# lenlisrt[:10]

In [63]:
sdf = test_df.drop(['text', 'ipa', 'ctext' ], axis=1)
sdf.head(2)

,row_id_column_name,fipa
0,0,bɪʃeʃ ot̪ɪt̪ʰɪ epʰem ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛd...
1,1,e nɪʲe bɪbɐd̪e d̪uɪ̯ hɐɟɐɾ pɔneɾo ʃɐleɾ d̪uɪ̯ ...


In [64]:
print(sdf.shape)
sdf.to_csv('/kaggle/working/sub2.csv', index=False)

(27228, 2)


In [65]:
sdf.fipa[0]

'bɪʃeʃ ot̪ɪt̪ʰɪ epʰem ɪkbɐl bɪn ɐnoʷɐɾ (dɔn) ɛdɪʃonɐl dɐt̪ɐɾektɔɾgems ɛnd spoɾts dɪpɐɾtoment oʷɐltɔn।'

In [66]:
# See a random sample from the test set 
rand_index = np.random.randint(0, len(test_df))
print(f'Ipa = {sdf.fipa[rand_index]}')

Ipa = ɐmɐd̪eɾ lokkʰo nɪɟed̪eɾ ʃeɾɐtɐ upohɐɾ d̪eʲɐ, ‘ bolecʰen t̪ɪnɪ।
